In [333]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns = 30
pd.options.display.float_format = '{:.2f}'.format

In [261]:
df = pd.read_csv(r'D:\Udemy\DataScience_12_Projects\# 3\Project_03_Materials\movies_metadata.csv', low_memory = False)
df

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.00,81.00,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.70,5415.00
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.00,104.00,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.90,2413.00
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.00,101.00,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.50,92.00
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.00,127.00,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.10,34.00
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.00,106.00,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.70,173.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,0.072051,/jldsYflnId4tTWPx8es3uzsB1I8.jpg,[],"[{'iso_3166_1': 'IR', 'name': 'Iran'}]",NaN,0.00,90.00,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.00,1.00
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,0.178241,/xZkmxsNmYXJbKVsTRLLx3pqGHx7.jpg,"[{'name': 'Sine Olivia', 'id': 19653}]","[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",2011-11-17,0.00,360.00,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.00,3.00
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,"[{'name': 'American World Pictures', 'id': 6165}]","[{'iso_3166_1': 'US', 'name': 'United States o...",2003-08-01,0.00,90.00,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of w

In [262]:
df.info()
# we have mixed data types, budget is object instead of float

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [263]:
df.genres[0]
# a nested string instead of list

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [264]:
# Thus we have a messy data set and a lot to clean up
# starting with dropping a few redundant or less useful columns
df.drop(['adult', 'imdb_id', 'original_title', 'video', 'homepage'], axis = 1, inplace = True)


In [265]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4494 non-null   object 
 1   budget                 45466 non-null  object 
 2   genres                 45466 non-null  object 
 3   id                     45466 non-null  object 
 4   original_language      45455 non-null  object 
 5   overview               44512 non-null  object 
 6   popularity             45461 non-null  object 
 7   poster_path            45080 non-null  object 
 8   production_companies   45463 non-null  object 
 9   production_countries   45463 non-null  object 
 10  release_date           45379 non-null  object 
 11  revenue                45460 non-null  float64
 12  runtime                45203 non-null  float64
 13  spoken_languages       45460 non-null  object 
 14  status                 45379 non-null  object 
 15  ta

### Handling stringified JSON columns (ex- genres here) 

In [266]:
import json
import ast # stands for 'abstract syntax trees'

In [267]:
json_col = ['belongs_to_collection', 'genres', 'production_companies', 'production_countries', 'spoken_languages']

In [268]:
json_col_1_0 = df.belongs_to_collection[0]
json_col_1_0
# given in string form (notice that keys have single quotes instead of double quotes)

"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}"

In [269]:
# json.loads(json_col_1_0) #loads deserializes the stringified json data to python object
# shows json decode error since property names (keys) are expected to be in double quotes

In [270]:
# putting them in double quotes
json.loads(json_col_1_0.replace("'", '"'))

{'id': 10194,
 'name': 'Toy Story Collection',
 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg',
 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}

In [271]:
# This can be done for the entire column
df.genres.apply(lambda x: json.loads(x.replace("'", '"')))

0        [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
1        [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
2        [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...
3        [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...
4                           [{'id': 35, 'name': 'Comedy'}]
                               ...                        
45461    [{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...
45462                        [{'id': 18, 'name': 'Drama'}]
45463    [{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...
45464                                                   []
45465                                                   []
Name: genres, Length: 45466, dtype: object

In [272]:
# However this can be done with ast library
df.genres.apply(ast.literal_eval)
# 'ast.literal_eval' evaluates python string and finds more appropriate dtypes for the string

0        [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
1        [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
2        [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...
3        [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...
4                           [{'id': 35, 'name': 'Comedy'}]
                               ...                        
45461    [{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...
45462                        [{'id': 18, 'name': 'Drama'}]
45463    [{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...
45464                                                   []
45465                                                   []
Name: genres, Length: 45466, dtype: object

In [273]:
# applying to all the columns
# df.loc[:, json_col].apply(ast.literal_eval, axis = 0)
# shows valueerror because of 0 somewhere (mixed dtypes)

In [274]:
# to solve this, we check whether each object is string or not and replace to np.nan if not!
df2 = df.loc[:, json_col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else np.nan)
df2

belongs_to_collection   nan
genres                  nan
production_companies    nan
production_countries    nan
spoken_languages        nan
dtype: float64

In [275]:
df.belongs_to_collection = df.belongs_to_collection.apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else np.nan)

In [276]:
df.belongs_to_collection

0        {'id': 10194, 'name': 'Toy Story Collection', ...
1                                                      NaN
2        {'id': 119050, 'name': 'Grumpy Old Men Collect...
3                                                      NaN
4        {'id': 96871, 'name': 'Father of the Bride Col...
                               ...                        
45461                                                  NaN
45462                                                  NaN
45463                                                  NaN
45464                                                  NaN
45465                                                  NaN
Name: belongs_to_collection, Length: 45466, dtype: object

In [277]:
df.genres = df.genres.apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else np.nan)

### How to flatten nested columns 

In [278]:
df.belongs_to_collection = df.belongs_to_collection.apply(lambda x: x['name'] if isinstance(x, dict) else np.nan)

In [279]:
df.belongs_to_collection

0                  Toy Story Collection
1                                   NaN
2             Grumpy Old Men Collection
3                                   NaN
4        Father of the Bride Collection
                      ...              
45461                               NaN
45462                               NaN
45463                               NaN
45464                               NaN
45465                               NaN
Name: belongs_to_collection, Length: 45466, dtype: object

In [280]:
df.belongs_to_collection.value_counts(dropna = False).head(20)

NaN                                       40975
The Bowery Boys                              29
Totò Collection                              27
Zatôichi: The Blind Swordsman                26
James Bond Collection                        26
The Carry On Collection                      25
Pokémon Collection                           22
Charlie Chan (Sidney Toler) Collection       21
Godzilla (Showa) Collection                  16
Dragon Ball Z (Movie) Collection             15
Charlie Chan (Warner Oland) Collection       15
Uuno Turhapuro                               15
Monster High Collection                      14
The Land Before Time Collection              14
George Carlin Comedy Collection              13
Sharpe Collection                            13
Friday the 13th Collection                   12
Sherlock Holmes (1939 series)                12
The Amityville Horror Collection             12
Johan Falk GSI Collection                    12
Name: belongs_to_collection, dtype: int6

In [281]:
df.head(3)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,Toy Story Collection,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.00,81.00,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.70,5415.00
1,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.00,104.00,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.90,2413.00
2,Grumpy Old Men Collection,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.00,101.00,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.50,92.00


In [282]:
# we cleaned the belongs_to_collection column
# let us now do the same with genre
df.genres[0]

[{'id': 16, 'name': 'Animation'},
 {'id': 35, 'name': 'Comedy'},
 {'id': 10751, 'name': 'Family'}]

In [283]:
df.genres = df.genres.apply(lambda x: "|".join(i['name'] for i in x))

In [284]:
df.genres[0]

'Animation|Comedy|Family'

In [285]:
df.genres.value_counts(dropna = False).head(10)

Drama                   5000
Comedy                  3621
Documentary             2723
                        2442
Drama|Romance           1301
Comedy|Drama            1135
Horror                   974
Comedy|Romance           930
Comedy|Drama|Romance     593
Drama|Comedy             532
Name: genres, dtype: int64

In [286]:
# replacing the missing 2442 values with np.nan
df.genres.replace("", np.nan, inplace = True)

In [287]:
# cleaning spoken_languages
df.spoken_languages[0]

"[{'iso_639_1': 'en', 'name': 'English'}]"

In [288]:
# so we first destringify it
df.spoken_languages = df.spoken_languages.apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else np.nan)
df.spoken_languages[0]

[{'iso_639_1': 'en', 'name': 'English'}]

In [289]:
df.spoken_languages = df.spoken_languages.apply(lambda x: '|'.join(i['name'] for i in x) if isinstance(x, list) else np.nan)

In [290]:
df.spoken_languages

0                 English
1        English|Français
2                 English
3                 English
4                 English
               ...       
45461               فارسی
45462                    
45463             English
45464                    
45465             English
Name: spoken_languages, Length: 45466, dtype: object

In [291]:
df.spoken_languages.value_counts(dropna = False).head(20)

English             22395
                     3952
Français             1853
日本語                  1289
Italiano             1218
Español               902
Pусский               807
Deutsch               762
English|Français      681
English|Español       572
हिन्दी                481
English|Deutsch       462
한국어/조선말               425
普通话                   347
English|Italiano      326
svenska               311
No Language           303
suomi                 275
Português             275
Polski                213
Name: spoken_languages, dtype: int64

In [292]:
df.spoken_languages.replace("", np.nan, inplace = True)

In [293]:
df.production_companies[0]

"[{'name': 'Pixar Animation Studios', 'id': 3}]"

In [294]:
# similarly for production_companies and production_countries
df.production_companies = df.production_companies.apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else np.nan)
df.production_companies

0           [{'name': 'Pixar Animation Studios', 'id': 3}]
1        [{'name': 'TriStar Pictures', 'id': 559}, {'na...
2        [{'name': 'Warner Bros.', 'id': 6194}, {'name'...
3        [{'name': 'Twentieth Century Fox Film Corporat...
4        [{'name': 'Sandollar Productions', 'id': 5842}...
                               ...                        
45461                                                   []
45462               [{'name': 'Sine Olivia', 'id': 19653}]
45463    [{'name': 'American World Pictures', 'id': 6165}]
45464                 [{'name': 'Yermoliev', 'id': 88753}]
45465                                                   []
Name: production_companies, Length: 45466, dtype: object

In [295]:
df.production_companies = df.production_companies.apply(lambda x: "|".join(i['name'] for i in x) if isinstance(x, list) else np.nan)
df.production_companies

0                                  Pixar Animation Studios
1        TriStar Pictures|Teitler Film|Interscope Commu...
2                              Warner Bros.|Lancaster Gate
3                   Twentieth Century Fox Film Corporation
4                Sandollar Productions|Touchstone Pictures
                               ...                        
45461                                                     
45462                                          Sine Olivia
45463                              American World Pictures
45464                                            Yermoliev
45465                                                     
Name: production_companies, Length: 45466, dtype: object

In [296]:
df.production_companies.replace("", np.nan, inplace = True)

In [297]:
df.production_countries[0]

"[{'iso_3166_1': 'US', 'name': 'United States of America'}]"

In [298]:
df.production_countries = df.production_countries.apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else np.nan)
df.production_countries

0        [{'iso_3166_1': 'US', 'name': 'United States o...
1        [{'iso_3166_1': 'US', 'name': 'United States o...
2        [{'iso_3166_1': 'US', 'name': 'United States o...
3        [{'iso_3166_1': 'US', 'name': 'United States o...
4        [{'iso_3166_1': 'US', 'name': 'United States o...
                               ...                        
45461               [{'iso_3166_1': 'IR', 'name': 'Iran'}]
45462        [{'iso_3166_1': 'PH', 'name': 'Philippines'}]
45463    [{'iso_3166_1': 'US', 'name': 'United States o...
45464             [{'iso_3166_1': 'RU', 'name': 'Russia'}]
45465     [{'iso_3166_1': 'GB', 'name': 'United Kingdom'}]
Name: production_countries, Length: 45466, dtype: object

In [299]:
df.production_countries = df.production_countries.apply(lambda x: "|".join(i['name'] for i in x) if isinstance(x, list) else np.nan)
df.production_countries

0        United States of America
1        United States of America
2        United States of America
3        United States of America
4        United States of America
                   ...           
45461                        Iran
45462                 Philippines
45463    United States of America
45464                      Russia
45465              United Kingdom
Name: production_countries, Length: 45466, dtype: object

In [300]:
df.production_countries.value_counts(dropna = False).head(20)

United States of America                   17851
                                            6282
United Kingdom                              2238
France                                      1654
Japan                                       1356
Italy                                       1030
Canada                                       840
Germany                                      749
India                                        735
Russia                                       735
United Kingdom|United States of America      569
South Korea                                  432
Spain                                        398
Canada|United States of America              365
Hong Kong                                    365
Australia                                    336
Sweden                                       332
Finland                                      271
France|Italy                                 235
Germany|United States of America             214
Name: production_cou

In [301]:
df.production_countries.replace("", np.nan, inplace = True)

In [302]:
df.isna().sum()
# notice that we have more missing values than when er originally read it
# hence identifying and labeling missing values (as np.nan or else) is also a part of data cleaning.

belongs_to_collection    40975
budget                       0
genres                    2442
id                           0
original_language           11
overview                   954
popularity                   5
poster_path                386
production_companies     11881
production_countries      6288
release_date                87
revenue                      6
runtime                    263
spoken_languages          3958
status                      87
tagline                  25054
title                        6
vote_average                 6
vote_count                   6
dtype: int64

In [303]:
df

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,Toy Story Collection,30000000,Animation|Comedy|Family,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,Pixar Animation Studios,United States of America,1995-10-30,373554033.00,81.00,English,Released,NaN,Toy Story,7.70,5415.00
1,NaN,65000000,Adventure|Fantasy|Family,8844,en,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,TriStar Pictures|Teitler Film|Interscope Commu...,United States of America,1995-12-15,262797249.00,104.00,English|Français,Released,Roll the dice and unleash the excitement!,Jumanji,6.90,2413.00
2,Grumpy Old Men Collection,0,Romance|Comedy,15602,en,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,Warner Bros.|Lancaster Gate,United States of America,1995-12-22,0.00,101.00,English,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.50,92.00
3,NaN,16000000,Comedy|Drama|Romance,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,Twentieth Century Fox Film Corporation,United States of America,1995-12-22,81452156.00,127.00,English,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.10,34.00
4,Father of the Bride Collection,0,Comedy,11862,en,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,Sandollar Productions|Touchstone Pictures,United States of America,1995-02-10,76578911.00,106.00,English,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.70,173.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,NaN,0,Drama|Family,439050,fa,Rising and falling between a man and woman.,0.072051,/jldsYflnId4tTWPx8es3uzsB1I8.jpg,NaN,Iran,NaN,0.00,90.00,فارسی,Released,Rising and falling between a man and woman,Subdue,4.00,1.00
45462,NaN,0,Drama,111109,tl,An artist struggles to finish his work while a...,0.178241,/xZkmxsNmYXJbKVsTRLLx3pqGHx7.jpg,Sine Olivia,Philippines,2011-11-17,0.00,360.00,NaN,Released,NaN,Century of Birthing,9.00,3.00
45463,NaN,0,Action|Drama|Thriller,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,American World Pictures,United States of America,2003-08-01,0.00,90.00,English,Released,A deadly game of wits.,Betrayal,3.80,6.00
45464,NaN,0,NaN,227506,en,"In a small town live two brothers, one a minis...",0.003503,/aorBPO7ak8e8iJKT5OcqYxU3jlK.jpg,Yermoliev,Russia,1917-10-21,0.00,87.00,NaN,Released,NaN,Satan Triumphant,0.00,0.00


### Cleaning Numerical columns

In [304]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4491 non-null   object 
 1   budget                 45466 non-null  object 
 2   genres                 43024 non-null  object 
 3   id                     45466 non-null  object 
 4   original_language      45455 non-null  object 
 5   overview               44512 non-null  object 
 6   popularity             45461 non-null  object 
 7   poster_path            45080 non-null  object 
 8   production_companies   33585 non-null  object 
 9   production_countries   39178 non-null  object 
 10  release_date           45379 non-null  object 
 11  revenue                45460 non-null  float64
 12  runtime                45203 non-null  float64
 13  spoken_languages       41508 non-null  object 
 14  status                 45379 non-null  object 
 15  ta

In [305]:
# above budget should be float instead of object 
# df.budget.astype('float') may not work like in this case because of unclean data
df.budget = pd.to_numeric(df.budget, errors = 'coerce')

In [306]:
df.budget.value_counts(dropna = False)

0.00           36573
5000000.00       286
10000000.00      259
20000000.00      243
2000000.00       242
               ...  
1119000.00         1
640000.00          1
17900000.00        1
4696772.00         1
2620000.00         1
Name: budget, Length: 1224, dtype: int64

In [307]:
df.budget.replace(0, np.nan, inplace = True)

In [308]:
df.budget.value_counts(dropna = False)

nan            36576
5000000.00       286
10000000.00      259
20000000.00      243
2000000.00       242
               ...  
4490331.00         1
313411.00          1
1020000.00         1
37.00              1
9500.00            1
Name: budget, Length: 1223, dtype: int64

In [309]:
# Also for better readability we convert from basic to 'in millions'
df.budget = df.budget.div(1000000)
df.budget.value_counts(dropna = False)

nan      36576
5.00       286
10.00      259
20.00      243
2.00       242
         ...  
0.44         1
1.64         1
29.50        1
0.03         1
0.80         1
Name: budget, Length: 1223, dtype: int64

In [310]:
# similarly for revenue columns
df.revenue.value_counts(dropna = False)

0.00            38052
12000000.00        20
10000000.00        19
11000000.00        19
2000000.00         18
                ...  
245724603.00        1
12212123.00         1
8203235.00          1
25900.00            1
12032983.00         1
Name: revenue, Length: 6864, dtype: int64

In [311]:
df.revenue.replace(0, np.nan, inplace = True)
df.revenue = df.revenue.div(1000000)
df.revenue

0       373.55
1       262.80
2          nan
3        81.45
4        76.58
         ...  
45461      nan
45462      nan
45463      nan
45464      nan
45465      nan
Name: revenue, Length: 45466, dtype: float64

In [312]:
# finally we rename the columns
df.rename(columns = {"revenue":"revenue_musd", "budget":"budget_musd"}, inplace = True)

In [313]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4491 non-null   object 
 1   budget_musd            8890 non-null   float64
 2   genres                 43024 non-null  object 
 3   id                     45466 non-null  object 
 4   original_language      45455 non-null  object 
 5   overview               44512 non-null  object 
 6   popularity             45461 non-null  object 
 7   poster_path            45080 non-null  object 
 8   production_companies   33585 non-null  object 
 9   production_countries   39178 non-null  object 
 10  release_date           45379 non-null  object 
 11  revenue_musd           7408 non-null   float64
 12  runtime                45203 non-null  float64
 13  spoken_languages       41508 non-null  object 
 14  status                 45379 non-null  object 
 15  ta

In [314]:
# for runtime
df.runtime.value_counts(dropna = False)

90.00      2556
0.00       1558
100.00     1470
95.00      1412
93.00      1214
           ... 
238.00        1
316.00        1
258.00        1
780.00        1
1256.00       1
Name: runtime, Length: 354, dtype: int64

In [315]:
df.runtime.replace(0, np.nan, inplace = True)

In [316]:
# for id
df.id = pd.to_numeric(df.id, errors = 'coerce')

In [317]:
df.id.value_counts(dropna = False)
# id has many duplicate values

141971.00    3
nan          3
12600.00     2
18440.00     2
42495.00     2
            ..
41244.00     1
10518.00     1
55971.00     1
91070.00     1
1408.00      1
Name: id, Length: 45434, dtype: int64

In [318]:
# similarly we can change 0 to np.nan for many columns
df.vote_count.value_counts(dropna = False)

1.00       3264
2.00       3132
0.00       2899
3.00       2787
4.00       2480
           ... 
1085.00       1
4461.00       1
1539.00       1
2487.00       1
3086.00       1
Name: vote_count, Length: 1821, dtype: int64

In [319]:
df.vote_average.value_counts(dropna = False)

0.00    2998
6.00    2468
5.00    2001
7.00    1886
6.50    1722
        ... 
9.10       3
9.60       1
0.70       1
9.80       1
1.10       1
Name: vote_average, Length: 93, dtype: int64

In [320]:
# movies that have both vote_average and vote_count as 0
df.loc[df.vote_count == 0, 'vote_average']
# All of them

83      0.00
107     0.00
126     0.00
132     0.00
137     0.00
        ... 
45432   0.00
45434   0.00
45452   0.00
45464   0.00
45465   0.00
Name: vote_average, Length: 2899, dtype: float64

In [321]:
# hence we change them to missing values
df.loc[df.vote_count == 0, 'vote_average'] = np.nan

### Cleaning columns with datetime

In [322]:
df.columns

Index(['belongs_to_collection', 'budget_musd', 'genres', 'id',
       'original_language', 'overview', 'popularity', 'poster_path',
       'production_companies', 'production_countries', 'release_date',
       'revenue_musd', 'runtime', 'spoken_languages', 'status', 'tagline',
       'title', 'vote_average', 'vote_count'],
      dtype='object')

In [323]:
df.release_date

0        1995-10-30
1        1995-12-15
2        1995-12-22
3        1995-12-22
4        1995-02-10
            ...    
45461           NaN
45462    2011-11-17
45463    2003-08-01
45464    1917-10-21
45465    2017-06-09
Name: release_date, Length: 45466, dtype: object

In [355]:
df.release_date = pd.to_datetime(df.release_date, errors = 'coerce')
df.release_date

TypeError: 'Series' object is not callable

In [ ]:
pd.to_datetime.value_counts(dropna = False)

In [335]:
# let us clean the data for each column, wherever required
df.original_language.value_counts(dropna = False).head(20)

en    32269
fr     2438
it     1529
ja     1350
de     1080
es      994
ru      826
hi      508
ko      444
zh      409
sv      384
pt      316
cn      313
fi      297
nl      248
da      225
pl      219
tr      150
cs      130
el      113
Name: original_language, dtype: int64

In [336]:
df.overview.value_counts(dropna = False).head(20)

NaN                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               954
No overview found.                                                                                                                                                                                                                                                                                                                                                                                                                

In [337]:
df.overview.replace('No overview found.', np.nan, inplace = True)

In [338]:
df.overview.replace('No overview ', np.nan, inplace = True)

In [339]:
df.overview.replace('No movie overview available.', np.nan, inplace = True)

In [340]:
df.overview.replace('No overview yet.', np.nan, inplace = True)

In [341]:
df.overview.replace(" ", np.nan, inplace = True)

In [342]:
df.overview.value_counts(dropna = False).head(20)

NaN                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   1097
No Overview                                                                                                                                                                                                                                                                                                                                                                  

In [343]:
df.tagline.value_counts(dropna = False).head(20)

NaN                                                                                                                      25054
Based on a true story.                                                                                                       7
Trust no one.                                                                                                                4
-                                                                                                                            4
Be careful what you wish for.                                                                                                4
There are two sides to every love story.                                                                                     3
Know Your Enemy                                                                                                              3
Which one is the first to return - memory or the murderer?                                                     

In [344]:
df.tagline.replace("-", np.nan, inplace = True)

### Remove Duplicates

In [345]:
# Use method 'duplicated'
df[df.duplicated(subset = 'id', keep = False)].sort_values(by = 'id')

,belongs_to_collection,budget_musd,genres,id,original_language,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue_musd,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
33826,NaN,30.00,Comedy|Crime|Drama|Romance|Thriller,4912.00,en,"Television made him famous, but his biggest hi...",7.645827,/o3Im9nPLAgtlw1j2LtpMebAotSe.jpg,Miramax Films|Allied Filmmakers|Mad Chance,United States of America,2002-12-30,33.01,113.00,English,Released,Some things are better left top secret.,Confessions of a Dangerous Mind,6.60,281.00
5865,NaN,30.00,Comedy|Crime|Drama|Romance|Thriller,4912.00,en,"Television made him famous, but his biggest hi...",11.331072,/o3Im9nPLAgtlw1j2LtpMebAotSe.jpg,Miramax Films|Allied Filmmakers|Mad Chance,United States of America,2002-12-30,33.01,113.00,English,Released,Some things are better left top secret.,Confessions of a Dangerous Mind,6.60,281.00
9165,NaN,nan,Crime|Drama|Thriller,5511.00,fr,Hitman Jef Costello is a perfectionist who alw...,9.091288,/cvNW8IXigbaMNo4gKEIps0NGnhA.jpg,Fida cinematografica|Compagnie Industrielle et...,France|Italy,1967-10-25,0.04,105.00,Français,Released,There is no solitude greater than that of the ...,Le Samouraï,7.90,187.00
7345,NaN,nan,Crime|Drama|Thriller,5511.00,fr,Hitman Jef Costello is a perfectionist who alw...,9.091288,/cvNW8IXigbaMNo4gKEIps0NGnhA.jpg,Fida cinematografica|Compagnie Industrielle et...,France|Italy,1967-10-25,0.04,105.00,Français,Released,There is no solitude greater than that of the ...,Le Samouraï,7.90,187.00
44821,Pokémon Collection,16.00,Adventure|Fantasy|Animation|Action|Family,10991.00,ja,When Molly Hale's sadness of her father's disa...,6.480376,/5ILjS6XB5deiHop8SXPsYxXWVPE.jpg,TV Tokyo|4 Kids Entertainment|Nintendo|Pikachu...,Japan,2000-07-08,68.41,93.00,English,Released,Pokémon: Spell of the Unknown,Pokémon: Spell of the Unknown,6.00,144.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40040,NaN,0.98,Drama|Fantasy,298721.00,th,"In a hospital, ten soldiers are being treated ...",2.535419,/5GasjPRAy5rlEyDOH7MeOyxyQGX.jpg,"Match Factory, The|Louverture Films|Tordenfilm...",United Kingdom|United States of America|France...,2015-09-02,nan,122.00,English|ภาษาไทย,Released,NaN,Cemetery of Splendour,4.40,50.00
33184,NaN,0.98,Drama|Fantasy,298721.00,th,"In a hospital, ten soldiers are being treated ...",2.535419,/5GasjPRAy5rlEyDOH7MeOyxyQGX.jpg,"Match Factory, The|Louverture Films|Tordenfilm...",United Kingdom|United States of America|France...,2015-09-02,nan,122.00,English|ภาษาไทย,Released,NaN,Cemetery of Splendour,4.40,50.00
19730,NaN,nan,Carousel Productions|Vision View Entertainment...,nan,104.0,Released,NaN,Midnight Man,NaN,NaN,1,nan,nan,NaN,NaN,NaN,NaN,nan,nan
29503,NaN,nan,Aniplex|GoHands|BROSTA TV|Mardock Scramble Pro...,nan,68.0,Released,NaN,Mardock Scramble: The Third Exhaust,NaN,NaN,12,nan,nan,NaN,NaN,NaN,NaN,nan,nan


In [346]:
df.drop_duplicates(subset = 'id', inplace = True)

In [347]:
df[df.duplicated(subset = 'id', keep = False)].sort_values(by = 'id')

,belongs_to_collection,budget_musd,genres,id,original_language,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue_musd,runtime,spoken_languages,status,tagline,title,vote_average,vote_count


In [348]:
# no duplicates anymore
df.id.value_counts(dropna = False)

287281.00    1
42323.00     1
42612.00     1
6473.00      1
38225.00     1
            ..
37942.00     1
64572.00     1
103751.00    1
255885.00    1
1408.00      1
Name: id, Length: 45434, dtype: int64

### Handling missing values and removing observations

In [349]:
# Three ways to handle missing values: Leave them as it is, Delete them (when they have say more than a few missing values), 
# replace them with appropriate values (Typically done in ML) after EDA and embedded with preprocessing steps

In [350]:
df.isna().sum()

belongs_to_collection    40946
budget_musd              36554
genres                    2442
id                           1
original_language           11
overview                  1097
popularity                   4
poster_path                386
production_companies     11872
production_countries      6283
release_date                87
revenue_musd             38036
runtime                   1819
spoken_languages          3954
status                      85
tagline                  25037
title                        4
vote_average              2900
vote_count                   4
dtype: int64

In [351]:
# movies with no title
df[df.title.isna()]

,belongs_to_collection,budget_musd,genres,id,original_language,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue_musd,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
19729,NaN,nan,Action|Thriller|Drama,82663.00,en,British soldiers force a recently captured IRA...,NaN,NaN,NaN,NaN,NaN,nan,nan,NaN,NaN,NaN,NaN,nan,nan
19730,NaN,nan,Carousel Productions|Vision View Entertainment...,nan,104.0,Released,NaN,Midnight Man,NaN,NaN,1,nan,nan,NaN,NaN,NaN,NaN,nan,nan
29502,Mardock Scramble Collection,nan,Animation|Science Fiction,122662.00,ja,Third film of the Mardock Scramble series.,NaN,NaN,NaN,NaN,NaN,nan,nan,NaN,NaN,NaN,NaN,nan,nan
35586,NaN,nan,TV Movie|Action|Horror|Science Fiction,249260.00,en,A group of skiers are terrorized during spring...,NaN,NaN,NaN,NaN,NaN,nan,nan,NaN,NaN,NaN,NaN,nan,nan


In [352]:
# As seen above, these movies have typically many na values
# hence we can drop them
df.dropna(subset = ['title', 'id'], inplace = True)

In [353]:
df.isna().sum()

belongs_to_collection    40943
budget_musd              36550
genres                    2442
id                           0
original_language           11
overview                  1097
popularity                   0
poster_path                383
production_companies     11868
production_countries      6279
release_date                84
revenue_musd             38032
runtime                   1815
spoken_languages          3950
status                      81
tagline                  25033
title                        0
vote_average              2896
vote_count                   0
dtype: int64

In [354]:
# now we can convert id column to int (from float)
# a column with na values cannot have int type
df.id = df.id.astype('int')

In [358]:
# deleting values that have more na values than a certain level
df.notna().sum(axis = 1).value_counts().sort_values(ascending = False)

15    12524
16    11454
14     5422
17     4265
18     3859
13     3040
12     1891
19     1132
11     1021
10      513
9       182
8       103
7        20
6         4
dtype: int64

In [359]:
df[df.notna().sum(axis = 1) <= 6]

,belongs_to_collection,budget_musd,genres,id,original_language,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue_musd,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
18038,NaN,nan,NaN,344741,pl,NaN,0.0,NaN,NaN,NaN,NaN,nan,nan,NaN,Released,NaN,Pajęczarki,nan,0.00
20166,NaN,nan,NaN,139909,fi,NaN,0.000127,NaN,NaN,NaN,NaN,nan,nan,NaN,Released,NaN,Aurinkotuuli,nan,0.00
41718,NaN,nan,NaN,216550,de,NaN,0.0,NaN,NaN,NaN,NaN,nan,nan,NaN,Released,NaN,Dolpo Tulku - Heimkehr in den Himalaya,nan,0.00
44978,NaN,nan,NaN,398295,pt,NaN,0.000331,NaN,NaN,NaN,NaN,nan,nan,NaN,Released,NaN,Mundo Cão,nan,0.00


In [362]:
df.dropna(thresh = 10, inplace = True)

In [366]:
df.notna().sum(axis = 1).value_counts().sort_values(ascending = False)

15    12524
16    11454
14     5422
17     4265
18     3859
13     3040
12     1891
19     1132
11     1021
10      513
dtype: int64

### Final Cleaning Steps

In [367]:
df.status.value_counts()

Released           44694
Rumored              226
Post Production       98
In Production         20
Planned               15
Canceled               2
Name: status, dtype: int64

In [372]:
df = df.loc[df.status == 'Released'].copy()
df

,belongs_to_collection,budget_musd,genres,id,original_language,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue_musd,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,Toy Story Collection,30.00,Animation|Comedy|Family,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,Pixar Animation Studios,United States of America,1995-10-30,373.55,81.00,English,Released,NaN,Toy Story,7.70,5415.00
1,NaN,65.00,Adventure|Fantasy|Family,8844,en,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,TriStar Pictures|Teitler Film|Interscope Commu...,United States of America,1995-12-15,262.80,104.00,English|Français,Released,Roll the dice and unleash the excitement!,Jumanji,6.90,2413.00
2,Grumpy Old Men Collection,nan,Romance|Comedy,15602,en,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,Warner Bros.|Lancaster Gate,United States of America,1995-12-22,nan,101.00,English,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.50,92.00
3,NaN,16.00,Comedy|Drama|Romance,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,Twentieth Century Fox Film Corporation,United States of America,1995-12-22,81.45,127.00,English,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.10,34.00
4,Father of the Bride Collection,nan,Comedy,11862,en,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,Sandollar Productions|Touchstone Pictures,United States of America,1995-02-10,76.58,106.00,English,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.70,173.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,NaN,nan,Drama|Family,439050,fa,Rising and falling between a man and woman.,0.072051,/jldsYflnId4tTWPx8es3uzsB1I8.jpg,NaN,Iran,NaN,nan,90.00,فارسی,Released,Rising and falling between a man and woman,Subdue,4.00,1.00
45462,NaN,nan,Drama,111109,tl,An artist struggles to finish his work while a...,0.178241,/xZkmxsNmYXJbKVsTRLLx3pqGHx7.jpg,Sine Olivia,Philippines,2011-11-17,nan,360.00,NaN,Released,NaN,Century of Birthing,9.00,3.00
45463,NaN,nan,Action|Drama|Thriller,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,American World Pictures,United States of America,2003-08-01,nan,90.00,English,Released,A deadly game of wits.,Betrayal,3.80,6.00
45464,NaN,nan,NaN,227506,en,"In a small town live two brothers, one a minis...",0.003503,/aorBPO7ak8e8iJKT5OcqYxU3jlK.jpg,Yermoliev,Russia,1917-10-21,nan,87.00,NaN,Released,NaN,Satan Triumphant,nan,0.00


In [373]:
# now we can drop this column
df = df.drop('status', axis = 1)

In [374]:
df.columns

Index(['belongs_to_collection', 'budget_musd', 'genres', 'id',
       'original_language', 'overview', 'popularity', 'poster_path',
       'production_companies', 'production_countries', 'release_date',
       'revenue_musd', 'runtime', 'spoken_languages', 'tagline', 'title',
       'vote_average', 'vote_count'],
      dtype='object')

In [376]:
# Adjusting columns
col = ['id', 'title', 'tagline', 'release_date', 'genres', 'belongs_to_collection', 'budget_musd', 'revenue_musd', 
       'original_language', 'overview', 'popularity', 'poster_path',
       'production_companies', 'production_countries', 
       'runtime', 'spoken_languages',
       'vote_average', 'vote_count']

In [377]:
df = df.loc[:, col]

In [378]:
df.head(3)

,id,title,tagline,release_date,genres,belongs_to_collection,budget_musd,revenue_musd,original_language,overview,popularity,poster_path,production_companies,production_countries,runtime,spoken_languages,vote_average,vote_count
0,862,Toy Story,NaN,1995-10-30,Animation|Comedy|Family,Toy Story Collection,30.00,373.55,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,Pixar Animation Studios,United States of America,81.00,English,7.70,5415.00
1,8844,Jumanji,Roll the dice and unleash the excitement!,1995-12-15,Adventure|Fantasy|Family,NaN,65.00,262.80,en,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,TriStar Pictures|Teitler Film|Interscope Commu...,United States of America,104.00,English|Français,6.90,2413.00
2,15602,Grumpier Old Men,Still Yelling. Still Fighting. Still Ready for...,1995-12-22,Romance|Comedy,Grumpy Old Men Collection,nan,nan,en,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,Warner Bros.|Lancaster Gate,United States of America,101.00,English,6.50,92.00


In [379]:
# adjust the index to the new cleaned data
df.reset_index(drop = True, inplace = True) 

In [389]:
df.poster_path[0]

"<imag_src='http://image.tmdb.org/t/p/w185//rhIRbceoE9lR4veEXuwCC2wARtG.jpg' style=height:100px;'>"

In [387]:
base_poster_url = 'http://image.tmdb.org/t/p/w185/'
df.poster_path = "<imag_src='" + base_poster_url + df2.poster_path + "' style=height:100px;'>"

In [388]:
df.poster_path[0]

"<imag_src='http://image.tmdb.org/t/p/w185//rhIRbceoE9lR4veEXuwCC2wARtG.jpg' style=height:100px;'>"

In [390]:
df.to_csv('movie_clean.csv', index = False)

In [392]:
# check the clean csv
pd.read_csv('movie_clean.csv').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44694 entries, 0 to 44693
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     44694 non-null  int64  
 1   title                  44694 non-null  object 
 2   tagline                20284 non-null  object 
 3   release_date           44660 non-null  object 
 4   genres                 42586 non-null  object 
 5   belongs_to_collection  4463 non-null   object 
 6   budget_musd            8854 non-null   float64
 7   revenue_musd           7385 non-null   float64
 8   original_language      44684 non-null  object 
 9   overview               43746 non-null  object 
 10  popularity             44694 non-null  float64
 11  poster_path            44333 non-null  object 
 12  production_companies   33356 non-null  object 
 13  production_countries   38835 non-null  object 
 14  runtime                43182 non-null  float64
 15  sp

In [386]:
df2 = pd.read_csv(r'D:\Udemy\DataScience_12_Projects\# 3\Project_03_Materials\movies_metadata.csv', low_memory = False)
df2.poster_path[0]

'/rhIRbceoE9lR4veEXuwCC2wARtG.jpg'